## Shannon scratches - see 10.1-3

In [3]:
# Dependencies
import pandas as pd
from pathlib import Path
# Python SQL toolkit and Object Relational Mapper
import sqlalchemy
from sqlalchemy.orm import Session
from sqlalchemy import create_engine
from sqlalchemy import func


In [ ]:
# Dependencies
# ----------------------------------
# Imports the method used for connecting to DBs
from sqlalchemy import create_engine

# Imports the methods needed to abstract classes into tables
from sqlalchemy.ext.declarative import declarative_base

# Allow us to declare column types
from sqlalchemy import Column, Integer, String, Float 

In [ ]:
# Create Database Connection
# ----------------------------------
# Establish Connection
engine = create_engine("sqlite:///surfer.sqlite")
conn = engine.connect()

In [ ]:
# Create a "Metadata" Layer That Abstracts our SQL Database
# ----------------------------------
# Create (if not already in existence) the tables associated with our classes.
Base.metadata.create_all(engine)

In [ ]:
# Create a Session Object to Connect to DB
# ----------------------------------
# Session is a temporary binding to our DB
from sqlalchemy.orm import Session
session = Session(bind=engine)

In [ ]:
# Use this to clear out the db
# ----------------------------------
Base.metadata.drop_all(engine)

In [5]:
# Jordan said to use this code:
for row in session.query(Demographics.id, Demographics.location).limit(5).all():
    print(row)

In [ ]:
# OR from sqlalchemy import create_engine
import pandas as pd

# Define the SQLite database file
db_file = "your_database_file.db"

# Create an SQLAlchemy engine
engine = create_engine(f"sqlite:///{db_file}")

# Test the connection
with engine.connect() as connection:
    result = connection.execute("SELECT sqlite_version();")
    print(result.fetchone())

In [ ]:
# Query to fetch the data
query = "SELECT * FROM flight_data;"

# Load data into a pandas DataFrame
df = pd.read_sql(query, engine)

# Display the DataFrame
print(df.head())

In [ ]:
total_arrivals_ranking = df[['carrier_name', 'total_arrivals']].sort_values(by='total_arrivals', ascending=False)
print(total_arrivals_ranking)
